# Preprocessing

In [1]:
import copy
import math
import random
from collections import OrderedDict, defaultdict

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import torchvision.models as models
import torchvision
from torch.utils.data import DataLoader

from torchvision.datasets import *
from torchvision.transforms import *


no_cuda = False
use_gpu = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

c:\Users\胡家豪\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

batch_size = 32

#Dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

#Dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Create NN model

In [3]:
class ToyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.backbone = nn.Sequential(
      nn.Linear(28*28, 120, bias=False),
      nn.ReLU(),
      nn.Linear(120, 84, bias=False),
      nn.ReLU(),
      nn.Linear(84, 10, bias=False)
    )

  def forward(self, x):
    x = x.view(-1, 28 * 28) #transform 28*28 figure to 784 vector
    x = self.backbone(x)
    return x

FP32_model = ToyModel()
print(FP32_model)

ToyModel(
  (backbone): Sequential(
    (0): Linear(in_features=784, out_features=120, bias=False)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=False)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=False)
  )
)


In [4]:
#train model
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  #Set the model to train mode
  model.train()
  for batch, (x, y) in enumerate(dataloader):
    if use_gpu:
      x, y = x.cuda(), y.cuda()
    optimizer.zero_grad()
    #forward
    pred = model(x)

    #loss
    loss = loss_fn(pred, y)

    #backward
    loss.backward()

    #optimize
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(x)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  #set model to evaluate mode
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for x, y in dataloader:
      if use_gpu:
        x, y = x.cuda(), y.cuda()
      pred = model(x)
      test_loss = loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item() #calculate accuracy
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
learning_rate = 1e-3
epochs = 3
loss_fn = nn.CrossEntropyLoss() #define loss function
optimizer = torch.optim.SGD(FP32_model.parameters(), lr=learning_rate, momentum=0.9)  #define optimizer

FP32_model.to(device) #let model on GPU

ToyModel(
  (backbone): Sequential(
    (0): Linear(in_features=784, out_features=120, bias=False)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=False)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=False)
  )
)

In [6]:
#Training
for i in range(epochs):
  print(f"Epoch {i+1}\n-------------------------------")
  train_loop(train_loader, FP32_model, loss_fn, optimizer)
  test_loop(test_loader, FP32_model, loss_fn)

Epoch 1
-------------------------------
loss: 2.333277  [   32/60000]
loss: 2.005216  [ 3232/60000]
loss: 1.397346  [ 6432/60000]
loss: 0.994156  [ 9632/60000]
loss: 0.724711  [12832/60000]
loss: 0.928478  [16032/60000]
loss: 0.531580  [19232/60000]
loss: 0.938654  [22432/60000]
loss: 0.637252  [25632/60000]
loss: 0.638972  [28832/60000]
loss: 0.741736  [32032/60000]
loss: 0.702593  [35232/60000]
loss: 0.494755  [38432/60000]
loss: 0.825992  [41632/60000]
loss: 0.718040  [44832/60000]
loss: 0.641048  [48032/60000]
loss: 0.446608  [51232/60000]
loss: 0.655020  [54432/60000]
loss: 0.735440  [57632/60000]
Test Error: 
 Accuracy: 79.9%, Avg loss: 0.000865 

Epoch 2
-------------------------------
loss: 0.546462  [   32/60000]
loss: 0.444471  [ 3232/60000]
loss: 0.381041  [ 6432/60000]
loss: 0.347778  [ 9632/60000]
loss: 0.617674  [12832/60000]
loss: 0.536812  [16032/60000]
loss: 0.392261  [19232/60000]
loss: 0.430454  [22432/60000]
loss: 0.380433  [25632/60000]
loss: 0.552010  [28832/60000

# Quantization definition

####Question 1.####

Use
>$S=(r_{\mathrm{max}} - r_{\mathrm{min}}) / (q_{\mathrm{max}} - q_{\mathrm{min}})$

>$Z = q_{\mathrm{min}} - r_{\mathrm{min}} / S$

to calculate scale factor and zero point of a tensor


In [7]:
def get_scale_and_zero_point(fp32_tensor, bitwidth=8):
  q_min, q_max = -2**(bitwidth-1), 2**(bitwidth-1) - 1
  fp_min = fp32_tensor.min().item()
  fp_max = fp32_tensor.max().item()

  #####################################################

  scale = (fp_max-fp_min) / (q_max-q_min)
  zero_point = q_min-fp_min /scale

  #####################################################


  zero_point = round(zero_point)          #round
  zero_point = max(q_min, min(zero_point, q_max)) #clip

  return scale, int(zero_point)

####Question 2.####

Use $q=r/S + Z$ to quantize a tensor

In [8]:
def linear_quantize(fp32_tensor, bitwidth=8):
  q_min, q_max = -2**(bitwidth-1), 2**(bitwidth-1) - 1

  scale, zero_point = get_scale_and_zero_point(fp32_tensor)

  #####################################################

  q_tensor = torch.round( fp32_tensor/scale ) +zero_point

  #####################################################

  #clamp
  q_tensor = torch.clamp(q_tensor, q_min, q_max)
  return q_tensor, scale, zero_point

####Question 3.####

Use
> $q_{\mathrm{output}} = M * \mathrm{Linear}[q_{\mathrm{input}}, q_{\mathrm{weight}}] + Z_{\mathrm{output}}$

> $M = S_{\mathrm{input}} * S_{\mathrm{weight}} / S_{\mathrm{output}}$

to compute quantized linear operation

In [9]:
def quantized_linear(input, weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point, device, bitwidth=8, activation_bitwidth=8):
  input, weights = input.to(device), weights.to(device)

  #####################################################

  M = input_scale * weight_scale / output_scale
  output = torch.nn.functional.linear((input - input_zero_point ), (weights - weight_zero_point))
  output *= M
  output += output_zero_point

  #####################################################

  #clamp and round
  output = output.round().clamp(-2**(activation_bitwidth-1), 2**(activation_bitwidth-1)-1)

  return output

# Design quantized linear layer and preprocess

In [10]:
class QuantizedLinear(nn.Module):
  def __init__(self, weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point, bitwidth=8, activation_bitwidth=8):
    super().__init__()
    self.weights = weights
    self.input_scale, self.input_zero_point = input_scale, input_zero_point
    self.weight_scale, self.weight_zero_point = weight_scale, weight_zero_point
    self.output_scale, self.output_zero_point = output_scale, output_zero_point

    self.bitwidth = bitwidth
    self.activation_bitwidth = activation_bitwidth

  def forward(self, x):
    return quantized_linear(x, self.weights, self.input_scale, self.weight_scale, self.output_scale, self.input_zero_point, self.weight_zero_point, self.output_zero_point, device)
  def __repr__(self):
    return f"QuantizedLinear(in_channels={self.weights.size(1)}, out_channels={self.weights.size(0)})"

#Transform input data to correct integer range
class Preprocess(nn.Module):
  def __init__(self, input_scale, input_zero_point, activation_bitwidth=8):
    super().__init__()
    self.input_scale, self.input_zero_point = input_scale, input_zero_point
    self.activation_bitwidth = activation_bitwidth
  def forward(self, x):
    x = x / self.input_scale + self.input_zero_point
    return x

# Calibration

In [11]:
# add hook to record the min max value of the activation
input_activation = {}
output_activation = {}

#Define a hook to record the feature map of each layer
def add_range_recoder_hook(model):
    import functools
    def _record_range(self, x, y, module_name):
        x = x[0]
        input_activation[module_name] = x.detach()
        output_activation[module_name] = y.detach()

    all_hooks = []
    for name, m in model.named_modules():
        if isinstance(m, (nn.Linear, nn.ReLU)):
            all_hooks.append(m.register_forward_hook(
                functools.partial(_record_range, module_name=name)))
            print(_record_range.x)

    return all_hooks

hooks = add_range_recoder_hook(FP32_model)
sample_data = iter(train_loader).__next__()[0].to(device) #Use a batch of training data to calibrate
FP32_model(sample_data) #Forward to use hook

# remove hooks
for h in hooks:
    h.remove()


[<torch.utils.hooks.RemovableHandle object at 0x00000238AC70D2D0>, <torch.utils.hooks.RemovableHandle object at 0x00000238CA1DD010>]
[<torch.utils.hooks.RemovableHandle object at 0x00000238AC70D2D0>, <torch.utils.hooks.RemovableHandle object at 0x00000238CA1DD010>, <torch.utils.hooks.RemovableHandle object at 0x00000238CA1E2FD0>]
[<torch.utils.hooks.RemovableHandle object at 0x00000238AC70D2D0>, <torch.utils.hooks.RemovableHandle object at 0x00000238CA1DD010>, <torch.utils.hooks.RemovableHandle object at 0x00000238CA1E2FD0>, <torch.utils.hooks.RemovableHandle object at 0x00000238CA1E0250>]
[<torch.utils.hooks.RemovableHandle object at 0x00000238AC70D2D0>, <torch.utils.hooks.RemovableHandle object at 0x00000238CA1DD010>, <torch.utils.hooks.RemovableHandle object at 0x00000238CA1E2FD0>, <torch.utils.hooks.RemovableHandle object at 0x00000238CA1E0250>, <torch.utils.hooks.RemovableHandle object at 0x00000238ADF03C10>]


# Quantize model

In [12]:
#copy original model
quantized_model = copy.deepcopy(FP32_model)

#Record each layer in original model
quantized_backbone = []
i = 0

#Record input scale and zero point
input_scale, input_zero_point = get_scale_and_zero_point(input_activation["backbone.0"])
preprocess = Preprocess(input_scale, input_zero_point)
quantized_backbone.append(preprocess)

#Record Linear + ReLU of the model (except the last Linear)
while i < len(quantized_model.backbone) - 1:
  if isinstance(quantized_model.backbone[i], nn.Linear) and isinstance(quantized_model.backbone[i+1], nn.ReLU):
    linear = quantized_model.backbone[i]
    linear_name = f"backbone.{i}"
    relu = quantized_model.backbone[i + 1]
    relu_name = f"backbone.{i + 1}"

    #Use the calibration data to calculate scale and zero point of each layer
    input_scale, input_zero_point = get_scale_and_zero_point(input_activation[linear_name])
    output_scale, output_zero_point = get_scale_and_zero_point(output_activation[relu_name])
    quantized_weights, weight_scale, weight_zero_point = linear_quantize(linear.weight.data)

    quantizedLinear = QuantizedLinear(quantized_weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point)

    quantized_backbone.append(quantizedLinear)
    i += 2

#Record the last Linear layer
linear = quantized_model.backbone[4]
linear_name = f"backbone.4"
input_scale, input_zero_point = get_scale_and_zero_point(input_activation[linear_name])
output_scale, output_zero_point = get_scale_and_zero_point(output_activation[linear_name])
quantized_weights, weight_scale, weight_zero_point = linear_quantize(linear.weight.data)
quantizedLinear = QuantizedLinear(quantized_weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point)
quantized_backbone.append(quantizedLinear)


quantized_model.backbone = nn.Sequential(*quantized_backbone)

In [13]:
print(quantized_model)

ToyModel(
  (backbone): Sequential(
    (0): Preprocess()
    (1): QuantizedLinear(in_channels=784, out_channels=120)
    (2): QuantizedLinear(in_channels=120, out_channels=84)
    (3): QuantizedLinear(in_channels=84, out_channels=10)
  )
)


# Evaluate

In [14]:
test_loop(test_loader, FP32_model, loss_fn)

Test Error: 
 Accuracy: 84.2%, Avg loss: 0.000704 



In [15]:
test_loop(test_loader, quantized_model, loss_fn)

Test Error: 
 Accuracy: 84.2%, Avg loss: 0.002995 

